# 3D Transformations

## Packages:

In [587]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import numpy as np
import ipywidgets as widgets
import json

## Utilities:

In [588]:
def mesh2D(xlim, ylim, n=5): #Set up Meshgrid
    if isinstance(n, int):
        x = np.linspace(xlim[0],xlim[1],n, dtype = np.int16)
        y = np.linspace(ylim[0],ylim[1],n, dtype = np.int16)
    elif isinstance(n, list):
        x = np.linspace(xlim[0],xlim[1],n[0],dtype = np.int16)
        y = np.linspace(ylim[0],ylim[1],n[1], dtype = np.int16)
    else:
        raise Exception("Invalid Parameter")
        
    return np.meshgrid(x, y, sparse=True)

In [589]:
def normalize(v): #for Evec, Eval
    magnitude = np.sqrt(v[0]**2 + v[1]**2 + v[2]**2)
    if magnitude==0:
        raise ValueError("Zero vector cannot be normalized.")
    else:
        return v/magnitude

## Objects:

In [590]:
class Line: 
    def __init__(self, pointList):
        self.x = []
        self.y = []
        self.z = []
        
        for i in range(len(pointList)):
            self.x.append(pointList[i][0])
            self.y.append(pointList[i][1])
            self.z.append(pointList[i][2])
        
    def gObject(self, name='', color='rgb(210,64,0)'):
        lineObject = go.Scatter3d(name=name,
                                  mode="lines",
                                  x=self.x,
                                  y=self.y,
                                  z=self.z,
                                  line=dict(color=(color),
                                            width=7)
                                 )
        return lineObject

In [591]:
class Sphere:
    def __init__(self, radius=5, center=[0, 0, 0]):
        self.radius = radius
        self.center = center
        meshSize = 20
        theta = np.linspace(0,2*np.pi,meshSize)
        phi = np.linspace(0,np.pi,meshSize)
        self.x = radius*np.outer(np.cos(theta),np.sin(phi)) + center[0]
        self.y = radius*np.outer(np.sin(theta),np.sin(phi)) + center[1]
        self.z = radius*np.outer(np.ones(meshSize),np.cos(phi)) + center[2]
        
    def gObject(self, name='Sphere', color=[[0.0, 'rgb(0,62,116)'], [1.0, 'rgb(255,255,255)']]):
        sphere = go.Surface(name=name,
                            x=self.x.tolist(),
                            y=self.y.tolist(),
                            z=self.z.tolist(),
                            showscale=False,
                            opacity=0.7,
                            colorscale=color
                           )       
        return sphere

In [592]:
class Point:
    def __init__(self, position):
        self.position = np.array(position)
    
    def gObject(self, name='', color='rgb(0,62,116)'):
        point = go.Scatter3d(mode="markers",
                             name=name,
                             x=[self.position[0]],
                             y=[self.position[1]],
                             z=[self.position[2]],
                             marker=dict(color=color,
                                         size=7
                                        )
                            )
        return point

Orange: 'rgb(210,64,0)'
Dark Green: 'rgb(2,137,59)'
Imperial Blue: 'rgb(0,62,116)'
Pool Blue: 'rgb(2,161,205)'

## 3D Visualization

### Unit Vectors and a Sphere

In [593]:
data = []
for i in range(3):
    uvec = [0, 0, 0]
    uvec[i] = 1
    line = Line([[0, 0, 0], uvec])
    data.append(line.gObject('rgb(0,0,0)'))
    
# radius = 2*np.sqrt(3)
# sphere = Sphere(radius)
# data.append(sphere.gObject())

### Frames

In [594]:
theta1 = np.pi/2
t = np.linspace(0, theta1, 10, dtype = np.int16)
initialPosition = np.matrix([[2], [2], [2]])

In [595]:
def roXaxis(theta):
    M = np.matrix([[1, 0, 0],
                   [0, np.cos(theta), -np.sin(theta)], 
                   [0, np.sin(theta), np.cos(theta)]
                  ])
    return M

def roYaxis(theta):
    M = np.matrix([[np.cos(theta), 0, np.sin(theta)],
                   [0, 1, 0],
                   [-np.sin(theta), 0, np.cos(theta)]
                  ])
    return M

def roZaxis(theta):
    M = np.matrix([[np.cos(theta), -np.sin(theta), 0],
                   [np.sin(theta), np.cos(theta), 0],
                   [0, 0 ,1]
                  ])
    return M

In [596]:
def computeFrames(rotation, theta, point, frames):
    vecPoint = np.matrix([[point[0]],
                          [point[1]],
                          [point[2]]
                         ])
    t = np.linspace(0, theta, frames,dtype = np.int16)
    
    lineList = [point]
    output = []
    for i in t:
        newPoint = rotation(i)*vecPoint
        ptList = np.reshape(newPoint,(1,3)).tolist()[0]
        lineList.append(ptList)
        output.append([Point(ptList).gObject(), Line(lineList).gObject('rgb(210,64,0)')])
    return [output,ptList]

In [597]:
def computeCompositeFrames(rotation1,rotation2, theta, point, frames):
    vecPoint = np.matrix([[point[0]],
                          [point[1]],
                          [point[2]]
                         ])
    t = np.linspace(0, theta, frames)
    
    lineList = [point]
    output = []
    for i in t:
        newPoint = rotation1(i)*vecPoint
        ptList = np.reshape(newPoint,(1,3)).tolist()[0]
        lineList.append(ptList)
        output.append([Point(ptList).gObject(), Line(lineList).gObject('rgb(210,64,0)')])
    for j in t:
        newPoint2 = rotation2(j)*newPoint
        ptList = np.reshape(newPoint2,(1,3)).tolist()[0]
        lineList.append(ptList)
        output.append([Point(ptList).gObject(), Line(lineList).gObject('rgb(210,64,0)')])
        
    return output

In [598]:
# frameSize = 10
# frameList = computeFrames(roZaxis, np.pi/2, [2, 2, 2], frameSize)

# data.append(frameList[9][0])
# data.append(frameList[9][1])

# newPoint = [frameList[9][0].x[0],
#             frameList[9][0].y[0],
#             frameList[9][0].z[0]
#            ]

# frameList = computeFrames(roXaxis, np.pi/2, newPoint, frameSize)

# data.append(frameList[9][0])
# data.append(frameList[9][1])

# newPoint = [frameList[9][0].x[0],
#             frameList[9][0].y[0],
#             frameList[9][0].z[0]
#            ]

# frameList = computeFrames(roYaxis, -np.pi/2, newPoint, frameSize)

# data.append(frameList[9][0])
# data.append(frameList[9][1])

frameSize = 10
frameList = computeFrames(roZaxis, np.pi/2, [2, 2, 2], frameSize)

data.append(frameList[5][0])
data.append(frameList[5][1])

In [599]:
# fig=go.Figure(data=data)
# py.plot(fig)

In [600]:
# def rotationPlot(frame):
#     global data
#     data = []
    
#     for i in range(3):
#         uvec = [0, 0, 0]
#         uvec[i] = 1
#         line = Line([[0, 0, 0], uvec])
#         data.append(line.gObject('rgb(0,0,0)'))
        
#     radius = 2*np.sqrt(3)
#     sphere = Sphere(radius)
#     data.append(sphere.gObject())
    
#     initialPoint = [2, 2, 2]
    
#     data.append(Point(initialPoint).gObject('rgb(0,0,0)'))
    
#     frameSize = 10
#     frameList1 = computeCompositeFrames(roYaxis,roZaxis,
#                               np.pi/2,
#                               initialPoint, 
#                               frameSize)
#     frameList2 = computeCompositeFrames(roZaxis,roYaxis,
#                               np.pi/2,
#                               initialPoint, 
#                               frameSize)
    
#     data.append(frameList1[frame][0]) 
#     data.append(frameList1[frame][1])
#     data.append(frameList2[frame][0]) 
#     data.append(frameList2[frame][1])

#     py.iplot(data)
    
# widgets.interact(rotationPlot,frame=widgets.IntSlider(min=0,max=19,step=1,value=0,continuous_update=True) )

In [601]:
data = []
basisName = ["iVec", "jVec", "kVec"]
for i in range(3):
    uvec = [0., 0., 0.]
    uvec[i] = 1
    line = Line([[0., 0., 0.], uvec])
    data.append(line.gObject(basisName[i], 'rgb(0,0,0)'))
        
radius = 2*np.sqrt(3)
sphere = Sphere(radius)
data.append(sphere.gObject(name='Sphere'))
    
initialPoint = [2., 2., 2.]
    
data.append(Point(initialPoint).gObject('initial point', 'rgb(0,0,0)'))

print(data)

frame = []    
frameSize = 10
frameList1 = computeCompositeFrames(roYaxis,roZaxis,
                                    np.pi/2,
                                    initialPoint, 
                                    frameSize)
frameList2 = computeCompositeFrames(roZaxis,roYaxis,
                                    np.pi/2,
                                    initialPoint, 
                                    frameSize)
for i in range(frameSize):
    frame.append(dict(data=frameList1[i][0], name="frame_a %i" %i)) 
    frame.append(dict(data=frameList1[i][1], name="frame_b %i" %i))    

[{'type': 'scatter3d', 'name': 'iVec', 'mode': 'lines', 'x': [0.0, 1], 'y': [0.0, 0.0], 'z': [0.0, 0.0], 'line': {'color': 'rgb(0,0,0)', 'width': 7}}, {'type': 'scatter3d', 'name': 'jVec', 'mode': 'lines', 'x': [0.0, 0.0], 'y': [0.0, 1], 'z': [0.0, 0.0], 'line': {'color': 'rgb(0,0,0)', 'width': 7}}, {'type': 'scatter3d', 'name': 'kVec', 'mode': 'lines', 'x': [0.0, 0.0], 'y': [0.0, 0.0], 'z': [0.0, 1], 'line': {'color': 'rgb(0,0,0)', 'width': 7}}, {'type': 'surface', 'name': 'Sphere', 'x': [[0.0, 0.5701723860344272, 1.1247919557063155, 1.6487301329403299, 2.1276952500664197, 2.54862238725826, 2.900029749488411, 3.1723318599658588, 3.358101026955403, 3.4522699518456452, 3.4522699518456452, 3.358101026955403, 3.172331859965859, 2.9000297494884117, 2.5486223872582605, 2.12769525006642, 1.6487301329403306, 1.1247919557063162, 0.5701723860344277, 4.242300954899627e-16], [0.0, 0.5392788734529514, 1.0638476250332096, 1.5593973866463435, 2.012410852597363, 2.4105309964530943, 2.742898138510911,

In [602]:

layout = {"layout":{
    "xaxis":{
      "title":"X Axis"
    },
    "yaxis":{
      "title":"Y Axis",    },
    "zaxis":{
      "title":"Z Axis",    },
    "hovermode":"closest",
    "slider":{
      "visible":True,
      "plotlycommand":"animate",
      "args":[
        "slider.value",
        {
          "duration":400,
          "ease":"cubic-in-out"
        }
      ],
      "initialValue":"0",
      "values":[
        "0",
        "1",
        "2",
        "3",
        "4",
        "5",
        "6",
        "7",
        "8",
        "8",
        "9",
      ]
    },
    "updatemenus": [{
      "x": 0.1,
      "y": 0,
      "yanchor": "top",
      "xanchor": "right",
      "showactive": False,
      "direction": "left",
      "type": "buttons",
      "pad": {"t": 87, "r": 10},
      "buttons": [{
        "method": "animate",
        "args": [None, {
          "fromcurrent": True,
          "transition": {
            "duration": 300,
            "easing": "quadratic-in-out"
          },
          "frame": {
            "duration": 500,
            "redraw": False
          }
        }],
        "label": "Play"
      }, {
        "method": "animate",
        "args": [
          [None],
          {
            "mode": "immediate",
            "transition": {
              "duration": 0
            },
            "frame": {
              "duration": 0,
              "redraw": False
            }
          }
        ],
        "label": "Pause"
      }]
    }],
    "sliders": [{
      "active": 0,
      "steps": [{
        "label": "frame 0",
        "method": "animate",
        "args": [["frame 0"], {
            "ode": "immediate",
            "transition": {"duration": 300},
            "frame": {"duration": 300, "redraw": False}
        
          }
        ]
    }, {
        "label": "frame 1",
        "method": "animate",
        "args": [["frame 1"], {
            "mode": "immediate",
            "transition": {"duration": 300},
            "frame": {"duration": 300, "redraw": False}
          }
        ]
      }, {
        "label": "frame 2",
        "method": "animate",
        "args": [["frame 2"], {
            "mode": "immediate",
            "transition": {"duration": 300},
            "frame": {"duration": 300, "redraw": False}
          }
        ]
      }, {
        "label": "frame 3",
        "method": "animate",
        "args": [["frame 3"], {
            "mode": "immediate",
            "transition": {"duration": 300},
            "frame": {"duration": 300, "redraw": False}
          }
        ]
      }, {
        "label": "frame 4",
        "method": "animate",
        "args": [["frame 4"], {
            "mode": "immediate",
            "transition": {"duration": 300},
            "frame": {"duration": 300, "redraw": False}
          }
        ]
      }, {
        "label": "frame 5",
        "method": "animate",
        "args": [["frame 5"], {
            "mode": "immediate",
            "transition": {"duration": 300},
            "frame": {"duration": 300, "redraw": False}
          }
        ]
      }, {
        "label": "frame 6",
        "method": "animate",
        "args": [["frame 6"], {
            "mode": "immediate",
            "transition": {"duration": 300},
            "frame": {"duration": 300, "redraw": False}
          }
        ]
      }, {
        "label": "frame 7",
        "method": "animate",
        "args": [["frame 7"], {
            "mode": "immediate",
            "transition": {"duration": 300},
            "frame": {"duration": 300, "redraw": False}
          }
        ]
      }, {
        "label": "frame 8",
        "method": "animate",
        "args": [["frame 8"], {
            "mode": "immediate",
            "transition": {"duration": 300},
            "frame": {"duration": 300, "redraw": False}
          }
        ]
      }, {
        "label": "frame 9",
        "method": "animate",
        "args": [["frame 9"], {
            "mode": "immediate",
            "transition": {"duration": 300},
            "frame": {"duration": 300, "redraw": False}
          }
        ]
      }],
        
    "x": 0.1,
      "len": 0.9,
      "xanchor": "left",
      "y": 0,
      "yanchor": "top",
      "pad": {"t": 50, "b": 10},
      "currentvalue": {
        "visible": True,
        "prefix": "Year:",
        "xanchor": "right",
        "font": {
          "size": 20,
          "color": "#666"
        }
      },
      "transition": {
        "duration": 300,
        "easing": "cubic-in-out"
      }
    }]
  },
  "config": {
    "scrollzoom": True
  }
}
        

In [603]:
mainData = dict(data=data, frame=frame,layout = layout)

In [604]:
import json
with open("data_dk.json", 'w') as test_file:
    json.dump(mainData, test_file, ensure_ascii=False)